In [31]:
import os
import matplotlib.pyplot as plt
import cv2
from IPython.display import Video
import json
import numpy as np

ids_we_care_about = [404,196,172,243,133,129,170]

replica_root = '/datasets01/replica/061819/18_scenes'
habitat_semantic_json = os.path.join(replica_root, 'apartment_0', 'habitat', 'info_semantic.json')
#         habitat_semantic_json = os.path.join(self.sjr, scene + '_info_semantic.json')
#         print(f"Using habitat semantic json {habitat_semantic_json}")
with open(habitat_semantic_json, "r") as f:
    hsd = json.load(f)
    
# print(hsd['classes'])
id_to_class_dict = {}
for x in hsd['classes']:
    id_to_class_dict[x['id']] = x['name']


def get_maximal_relevant_id(x, traj_path):
    image_id = int(x.split('.')[0])
#     print(f'image_id {image_id}')
    
    seg = os.path.join(traj_path, 'seg')
    prop_path = os.path.join(seg, "{:05d}.npy".format(image_id))
    annot = np.load(prop_path).astype(np.uint32)
    
    ans = -1
    maxarea = 0
    for x in np.unique(annot):
        if x in ids_we_care_about:
            binary_mask = (annot == x).astype(np.int32)
            area = binary_mask.sum()
            if area > maxarea:
                maxarea = area
                ans = x
    return ans
    
    
from shutil import copyfile, rmtree
import json
import os
import matplotlib.pyplot as plt
import cv2

def acopyjson(a, b, r, og_ctr):
    with open(a, "r") as f:
        dat = json.load(f)
        
    dat2 = {}
    
    if os.path.isfile(b):
        with open(b, "r") as f:
            dat2 = json.load(f)
    
    ctr = og_ctr
    for x in range(r[0], r[1]+1):
        dat2.update({ctr: dat[str(x)]})
        ctr += 1
        
    with open(b, "w") as fp:
        json.dump(dat2, fp)

def acopydir(a, b, ext, r, og_ctr):
#     print(f'acopydir {r}')
    if not os.path.isdir(b):
        os.makedirs(b) 
    ctr = og_ctr
    for x in range(r[0], r[1]+1):
        f = "{:05d}{}".format(x, ext)
#         print(f)
        copyfile(os.path.join(a,f), os.path.join(b,"{:05d}{}".format(ctr, ext)))
        ctr += 1

def acopydir(a, b, ext, r, og_ctr):
#     print(f'acopydir {r}')
    if not os.path.isdir(b):
        os.makedirs(b)    
    ctr = og_ctr
    for x in range(r[0], r[1]+1):
        f = "{:05d}{}".format(x, ext)
#         print(f)
        copyfile(os.path.join(a,f), os.path.join(b,"{:05d}{}".format(ctr, ext)))
        ctr += 1
    
    
def save_traj(start, end, traj_dir, inst_id, traj_out, ctr):
    print(f'save_traj {start, end}, instance_id {inst_id}, traj_out {traj_out}')
    
    # copy rgb, seg, depth and data.json
    acopydir(os.path.join(traj_dir, 'rgb'), os.path.join(traj_out, 'rgb'), '.jpg', (start,end), ctr)
    acopydir(os.path.join(traj_dir, 'depth'), os.path.join(traj_out, 'depth'), '.npy', (start,end), ctr)
    acopydir(os.path.join(traj_dir, 'seg'), os.path.join(traj_out, 'seg'), '.npy', (start,end), ctr)
    acopyjson(os.path.join(traj_dir, 'data.json'), os.path.join(traj_out, 'data.json'), (start,end), ctr)
    
    return ctr + (end-start+1)
    

per_instance_thresh = 1

from collections import defaultdict

def mine_traj(traj_path, out_dir, tbd):
    print(f'mining trajectory {traj_path} ...')
    if os.path.isdir(traj_path):
        # visualize the rgb_dbg dir
        rgb = os.path.join(traj_path, 'rgb')
        imgs = os.listdir(rgb)

        img_id_map = []
        # 
        for x in range(len(imgs)):
            ncur = get_maximal_relevant_id(imgs[x], traj_path)
            img_id_map.append(ncur)
        
        print(img_id_map)
        assert len(img_id_map) == len(imgs)

        ctr = 0   
        x = 0
        while x < len(img_id_map):
            cur = img_id_map[x]
            if cur != -1 and tbd[cur] < per_instance_thresh:
                start = x
                while x < len(img_id_map) and img_id_map[x] == cur:
                    end = x
                    x += 1
                if abs(start - end) > 10: # should have atleast 10 frames
                    ctr = save_traj(start, end, traj_path, cur, out_dir, ctr)
                    print(f'saving {end - start + 1} frames, ctr {ctr}')
                    tbd[cur] += 1

            else:
                x += 1
        done = True
        for k, v in tbd.items():
            print(f'instance {k} exlpore {v} times')
            if v < per_instance_thresh:
                return False
        return True
      
def prep(traj, data_dir, out_dir, pref=''):
    if os.path.isdir(out_dir):
        print(f'rmtree {out_dir}')
        rmtree(out_dir)
    os.makedirs(out_dir)
    tbd = defaultdict(int)
    while not mine_traj(os.path.join(data_dir, str(traj), pref),  out_dir, tbd):
        traj += 1

    print(f'Mined {per_instance_thresh} trajectories for all instance ids!')

In [30]:
dd = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2'
od = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids_allinone_3'
prep(2, dd, od)

rmtree /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids_allinone_3
mining trajectory /checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_2/2/ ...
save_traj (2, 31), instance_id 404, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids_allinone_3
saving 30 frames, ctr 30
save_traj (74, 148), instance_id 243, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids_allinone_3
saving 75 frames, ctr 105
save_traj (289, 317), instance_id 196, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids_allinone_3
saving 29 frames, ctr 134
save_traj (373, 404), instance_id 129, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids_allinone_3
saving 32 frames, ctr 166
save_traj (797, 818), instance_id 170, traj_out /checkpoint/apratik/data/data/apartment_0/default/no_noise/instance_detection_ids

<img src=vid_path align="center">

In [32]:
active_data_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019'
active_out_dir = '/checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_detection_ids_allinone_auto'
prep(1, active_data_dir, active_out_dir, pref='activeonly')

rmtree /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/instance_detection_ids_allinone_auto
mining trajectory /checkpoint/apratik/data_devfair0187/apartment_0/straightline/no_noise/1633991019/1/activeonly ...
[170, 170, 170, 170, 170, 170, 170, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 404, 243, 243, 243, 243, 243, 243, 243, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 243, 172, 172, 172, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 133, 133, 133, 133, 196, 196, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 133, 133, 133, 133, 133